In [39]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import numpy_financial as npf

ModuleNotFoundError: No module named 'numpy_financial'

In [2]:
par = 1000


In [3]:



def bond_ytm(price, par, T, coup, freq=2, guess=0.05):
    freq = float(freq)
    periods = T*freq
    coupon = coup/100.*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y: sum([coupon/(1+y/freq)**(freq*t) for t in dt]) + par/(1+y/freq)**(freq*T) - price
        
        
        
    return optimize.newton(ytm_func, guess)


ytm = bond_ytm(1110, 1000, 10, 20, 2)
ytm

0.1762232257487831

In [4]:
def bond_price(par, T, ytm, coup, freq=2):
    freq = float(freq)
    periods = T*freq
    coupon = (coup/100)*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    price = sum([coupon/(1+ytm/freq)**(freq*t) for t in dt]) + \
            par/(1+ytm/freq)**(freq*T)
    return price


pv = bond_price(1000, 10, 0.15, 20)
pv

1254.8622839797981

In [5]:
def durations_explicit(c, ytm, t, n):
    """Parameters:
        c = coupon rate per period
        y = yield per period
        m = periods per year
        n = periods remaining"""
    if c==y: # Shorter explicit formula if coupon rate per period = yield per period
        macaulay_duration = ((1+y)/(m*y))*(1 - (1 / (1+y)**n))
        
        
       
        return macaulay_duration
    else:
        macaulay_duration = ((1+y) / (m*y)) - ( (1 + y + n*(c-y)) / ((m*c* ((1+y)**n - 1)) + m*y) )
       
       
       
        return macaulay_duration


In [6]:
def effective_duration(price, par, T, coup, freq=2, dy=0.01):
    ytm = bond_ytm(price, par, T, coup, freq)
    
    ytm_minus = ytm - dy   
    price_minus = bond_price(par, T, ytm_minus, coup, freq)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(par, T, ytm_plus, coup, freq)
    
    mduration = (price_minus-price_plus)/(2*price*dy)
    return mduration



In [7]:
def bond_convexity(price, par, T, coup, freq, dy=0.01):
    ytm = bond_ytm(price, par, T, coup, freq)

    ytm_minus = ytm - dy    
    price_minus = bond_price(par, T, ytm_minus, coup, freq)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(par, T, ytm_plus, coup, freq)
    
    convexity = (price_minus+price_plus-2*price)/(price*dy**2)
    return convexity

convexidade = bond_convexity(1110, 1000, 10, 50, 2)
convexidade

8.987809765474939

In [8]:
#Sistema para gerar titulos 
cdi_gen = np.random.uniform(0, 0.20)

cdif = [cdi_gen]

for i in range(9):
    cdi_gen = cdi_gen * (1 + np.random.uniform(-0.15, 0.15))
    cdif.append(cdi_gen)


t_anos = 10

anos = [pd.Timestamp(year=i+2024, month=1, day=1) for i in range(t_anos)]


cdi = pd.DataFrame({"CDI Futuro" : cdif, "Ano" : anos})

cdi['Ano'] = pd.to_datetime(cdi['Ano'])

cdi['Ano'] = cdi['Ano'].dt.year.astype(str)

cdi.set_index("Ano")

    
    

,CDI Futuro
Ano,
2024,0.196761
2025,0.204931
2026,0.231961
2027,0.213052
2028,0.240655
2029,0.257704
2030,0.275419
2031,0.283357
2032,0.245209


In [9]:
class bonds:
    def __init__(self):
        self.par_value = 1000
        self.coupon = np.random.randint(low=0, high= self.par_value*0.05)
        self.maturity = np.random.randint(2024, 2033)
        self.vencimento = self.maturity - 2023
        ano_maturidade = str(self.maturity)
        
        
        
        
        for index, row in cdi.iterrows():
            if row['Ano'] == ano_maturidade:
                self.ytm_init = row['CDI Futuro']
                break
        
        self.pv = bond_price(self.par_value, self.vencimento,self.ytm_init , self.coupon)
        self.efecdu = effective_duration(self.pv, self.par_value, self.vencimento, self.coupon)
    def risco_buy(self):
        teste = np.random.randint(low=0, high= 3)
        if teste == 0:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0, high= 0.01)
        elif teste == 1:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0.01, high= 0.05)
        elif teste == 2:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0.05, high= 0.1)

    def risco_sell(self):
        teste = np.random.randint(low=0, high= 3)
        if teste == 0:
            self.ytm_init = self.ytm_init - np.random.uniform(low=0, high= self.ytm_init*0.1)
        elif teste == 1:
            self.ytm_init = self.ytm_init - np.random.uniform(low=self.ytm_init*0.1, high= self.ytm_init*0.2)
        elif teste == 2:
            self.ytm_init = self.ytm_init - np.random.uniform(low=self.ytm_init*0.2, high= self.ytm_init*0.3)
        




In [10]:
def create_bonds():
    data = []
    for i in range(len(cdi["CDI Futuro"])):
        bond = bonds()
        bond.risco_buy()
        title = f"Title {i+1}"
        pv = round(bond.pv, 2)
        ytm = round(bond.ytm_init*100, 2)
        vencimento = (bond.vencimento)
        cupom = (bond.coupon)
        data.append({"Nome": title, "Valor Presente": pv, "YTM" : ytm, "Vencimento" : vencimento, "Cupom Semestral" : cupom})
    df = pd.DataFrame(data)
    return df

create_bonds()

,Nome,Valor Presente,YTM,Vencimento,Cupom Semestral
0,Title 1,124.71,26.23,9,0
1,Title 2,1115.87,20.09,1,33
2,Title 3,528.40,27.83,7,12
3,Title 4,1363.73,27.64,6,38
4,Title 5,1269.07,30.78,8,37
5,Title 6,224.96,35.08,7,2
6,Title 7,1011.51,22.48,1,21
7,Title 8,374.58,29.00,9,7
8,Title 9,892.53,31.08,7,24
9,Title 10,430.58,35.67,8,10
